# Multi-layer perceptron
**Important**: Change the kernel to *PROJECT_NAME local*. You can do this from the *Kernel* menu under *Change kernel*.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
import pickle
import os
from azure.storage.blob import BlockBlobService

In [2]:
from azureml.logging import get_azureml_logger
run_logger = get_azureml_logger()
run_logger.log('amlrealworld.timeseries.mlp','true')

In [3]:
model_name = "mlp"
aml_dir = os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']
ACCOUNT_NAME = "petcgexperimentstorage"
ACCOUNT_KEY = "C+ffy45PBcicHAxWiRbW9MnFR651A8xbiVe2wkyZgolznhFf70caTZmpWIJb2spV5YFl/LQB0ARUfU+AQx6w9g=="
CONTAINER_NAME = "energy-demand-demo"
block_blob_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)
train = None
list_of_blobs = block_blob_service.list_blobs(CONTAINER_NAME)
for each_blob in list_of_blobs:
    if ("part-00" in each_blob.name):
        block_blob_service.get_blob_to_path(CONTAINER_NAME, blob_name=each_blob.name, file_path='./nyc_demand.csv')
        train = pd.read_csv('./nyc_demand.csv')
        break
X = train.drop(['demand', 'timeStamp'], axis=1)

Due to the relatively long training times of neural networks, in this example we use one validation set in place of cross-validation. The validation set will be the final 25% of the training dataset. Training takes about 3 minutes.

In [4]:
len_train = int(len(X) * 0.75)
len_valid = len(X) - len_train
test_fold = [-1]*len_train + [0]*len_valid
ps = PredefinedSplit(test_fold)

In [5]:
regr = MLPRegressor(solver='lbfgs', verbose=True)

We first specify a list of network structures to try. Each tuple in this list specifies the number of hidden layers and the number of units in each. For example, (5,0) specifies a network with one hidden layer containing 5 units while (10,10) creates a network with 2 hidden layers, each with 10 units. We then perform grid search to test the various network structures and regularization parameters. 

In [6]:
hidden_layer_size = [(5,), (10,), (15,), (20,), (25,), (30,), (35,), (40,), (10,10), (20,20), (30,30), (40,40)]

In [7]:
param_grid = {'hidden_layer_sizes': hidden_layer_size,
             'alpha': [0.01, 0.1, 1.0, 10.0]}
regr_cv = GridSearchCV(estimator=regr,
            param_grid=param_grid,
            cv=ps,
            scoring='neg_mean_squared_error',
            verbose=2,
            n_jobs=-1)

In [8]:
regr_pipe = Pipeline([('regr_cv', regr_cv)])
regr_pipe.fit(X, y=train['demand'])

Fitting 1 folds for each of 48 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  3.1min finished


Pipeline(steps=[('regr_cv', GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
       error_score='raise',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1...*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=2))])

In [9]:
model_dir = "C:/Users/nelgoh/Desktop/Resources/Petronas/energy_demand_forecast/EnergyDemandForecast/outputs/models/"
with open(os.path.join(model_dir, model_name + '.pkl'), 'wb') as f:
    pickle.dump(regr_pipe, f)

Cross validation results

In [10]:
cv_results = pd.DataFrame(regr_pipe.named_steps['regr_cv'].cv_results_)
cv_results.sort_values(by='rank_test_score', inplace=True)
cv_results.head()

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_alpha,param_hidden_layer_sizes,params,rank_test_score,split0_test_score,split0_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
43,26.640935,0.030055,-6213.237438,-7331.204078,10,"(40,)","{'hidden_layer_sizes': (40,), 'alpha': 10.0}",1,-6213.237438,-7331.204078,0.0,0.0,0.0,0.0
46,31.849793,0.021012,-6544.191046,-7719.943560,10,"(30, 30)","{'hidden_layer_sizes': (30, 30), 'alpha': 10.0}",2,-6544.191046,-7719.943560,0.0,0.0,0.0,0.0
22,41.956168,0.037994,-6867.225439,-8183.795294,0.1,"(30, 30)","{'hidden_layer_sizes': (30, 30), 'alpha': 0.1}",3,-6867.225439,-8183.795294,0.0,0.0,0.0,0.0
4,19.996697,0.016999,-6870.441838,-7974.766059,0.01,"(25,)","{'hidden_layer_sizes': (25,), 'alpha': 0.01}",4,-6870.441838,-7974.766059,0.0,0.0,0.0,0.0
30,24.480177,0.022003,-6904.081338,-7956.523899,1,"(35,)","{'hidden_layer_sizes': (35,), 'alpha': 1.0}",5,-6904.081338,-7956.523899,0.0,0.0,0.0,0.0
